# Import Necessaries

In [ ]:
import numpy as np
import pygame

from enum import Enum 
import os 

In [ ]:
os.environ["SDL_VIDEODRIVER"] = "dummy" 

# Set Parameters

In [ ]:
# parameter to creating the matrix
width, height = 4 * 130, 4 * 130
rows, cols = 13, 13
square_size = width // cols 

maze_path = 'Maze.npy'

# Load Maze

In [ ]:
Maze = np.load(maze_path)

Maze

array([[0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1.],
       [1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0.],
       [0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0.],
       [1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1.],
       [0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1.],
       [1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1.],
       [1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

# Create Board

In [ ]:
class colors(Enum):

  # RGB colors
  red = (255, 0, 0)
  white = (255, 255, 255)
  black = (0, 0, 0)
  blue = (0, 0, 255)
  evenColor = (63, 63, 63)
  oddColor = (70, 70, 70)
  blockColor = (28, 18, 18)
  startColor = (58, 42, 10)
  endColor = (0, 244, 11)
  playerColor = (12, 100, 150)
  green = (0, 255, 0)
  selectionColor = (12, 232, 255)

In [ ]:
class Tile:

    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.contains_player = False
        self.isStart = False
        self.isGoal = False
        self.isBlocked = False

        if (x + y) % 2 == 0:
            self.color = colors.evenColor.value
        else:
            self.color = colors.oddColor.value

    def block(self):
        self.isBlocked = True
        self.color = colors.blockColor.value

    def unblock(self):
        self.isBlocked = False
        if (self.get_coordinates()[0] + self.get_coordinates()[1]) % 2 == 0:
            self.color = colors.evenColor.value
        else:
            self.color = colors.oddColor.value

    def get_coordinates(self):
        return self.x, self.y

    def set_color(self, color):
        self.color = color

    def get_color(self):
        return self.color

    def get_rect(self):
        return (
            self.get_coordinates()[0] * square_size, self.get_coordinates()[1] * square_size, square_size, square_size)

    def make_start(self):
        self.isStart = True
        self.color = colors.startColor.value

    def make_goal(self):
        self.isGoal = True
        self.color = colors.endColor.value

    def contains_player(self):
        return self.contains_player

    def set_player_here(self):
        self.contains_player = True
        self.color = colors.playerColor.value

    def is_inside_me(self, mousePos):
        rectangle = pygame.Rect(self.get_rect())
        return True if rectangle.collidepoint(mousePos[0], mousePos[1]) else False

    def is_blocked(self):
        return self.isBlocked


In [ ]:
class Board:
    def __init__(self, start, end):
        self.boardArray = []
        arraynump = Maze

        # initializing 2D matrix as board:
        for row in range(rows):
            self.boardArray.append([])
            for col in range(cols):
                self.boardArray[row].append(Tile(row, col))

        # locking tiles based on maze file :
        for i in range(rows):
            for j in range(cols):
                if arraynump[j][i] == 0:
                    self.boardArray[i][j].block()

        # setting positions :
        # setting start positions :
        self.start_pos = start['x'], start['y']
        (self.boardArray[start['x']][start['y']]).make_start()

        # setting player position like start position
        self.agent_pos = start['x'], start['y']
        (self.boardArray[start['x']][start['y']]).set_player_here()

        # setting goal position :
        self.goal_pos = end['x'], end['y']
        (self.boardArray[end['x']][end['y']]).make_goal()

    # draws the matrix:
    def draw_tiles(self, win):
        for row in range(rows):
            for col in range(cols):
                tile = self.boardArray[row][col]
                pygame.draw.rect(win, tile.get_color(), tile.get_rect())

    # draws the matrix and shows on screen - use this method to display your outcomes
    def draw_world(self, win):
        self.draw_tiles(win) 
        pygame.display.update()

    def get_current_state(self):
        return self.boardArray

    def get_agent_pos(self):
        return self.agent_pos

    # Updates the matrix values
    def update_board(self, new_state):
        self.boardArray = new_state

    def set_agent_pos(self, new_pos):

        self.agent_pos = new_pos['x'], new_pos['y']
        (self.boardArray[new_pos['x']][new_pos['y']]).set_player_here()

    def colorize(self, x, y, color):
        self.boardArray[x][y].set_color(color)


# Your Code (Agent)

In [ ]:
class Agent:
    def __init__(self, board):
        self.position = board.get_agent_pos()
        self.current_state = board.get_current_state()

    def get_position(self):
        return self.position

    def set_position(self, position, board):
        self.position = position
        board.set_agent_pos(position)
        board.update_board(self.current_state)

    def percept(self, board):
        # perception :
        # sets the current state
        # Use get_current_state function to get the maze matrix. - make your state
        self.current_state = board.get_current_state()

        pass

    def move(self, direction, board):
        # make your next move based on your perception
        # check if the move destination is not blocked
        # if not blocked , move to destination - set new position
        # if you want to show visited tiles, use red color.
        # something like :
        current_pos = self.get_position()
        x, y = current_pos[0], current_pos[1]
        board.colorize(x, y, colors.red.value)
        self.set_position(self.get_position() + direction)

        pass

    @staticmethod
    def get_actions():
        actions = []
        # returns a list of valid actions
        return actions

    def bfs(self, environment):
        self.percept(environment)
        # now go on !
        
        # use green color and colorize method to show the path. 
        
        pass

    def dfs(self, environment):
        pass

    def a_star(self, environment):
        pass


# Main

In [ ]:
# initialize:
FPS = 60
pygame.init()
WIN = pygame.display.set_mode((width, height))
pygame.display.set_caption("Search Game")

# setting start and end point :
start = {'x': 6, 'y': 0}
end = {'x': 12, 'y': 0}

gameBoard = Board(start, end)
agent = Agent(gameBoard)

def main():
    run = True
    clock = pygame.time.Clock()
    WIN.fill(colors.black.value)

    while run:
        clock.tick(FPS)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                
        
            # pos = pygame.mouse.get_pos()  # gets the current mouse coords
            # if event.type == pygame.MOUSEBUTTONDOWN:
            #     for i in range(rows):
            #         for j in range(cols):
            #             rect = gameBoard.boardArray[i][j]
            #             if rect.is_inside_me(pos):
            
            #                 if event.button == 1:
            #                     gameBoard.boardArray[i][j].block()
            #                 if event.button == 3:
            #                     gameBoard.boardArray[i][j].unblock()

        gameBoard.draw_world(WIN)

    pygame.quit()

In [ ]:
main()